In [ ]:
import numpy as np
import glob
import os
import pyklip.fm as fm
from astropy.io import fits
import pyklip.fmlib.extractSpec as es
from pyklip.instruments import CHARIS as charis
import pyklip.parallelized as parallelized
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
image_files = glob.glob("images_path")
image_files.sort()

In [ ]:
#dataset = charis.CHARISData(image_files,skipslices=[6,14,15,21])
dataset = charis.CHARISData(image_files, update_hdrs=True)

In [ ]:
# generate psf cube
boxrad = 10
dataset.generate_psfs(boxrad)

In [ ]:
# obtain number of files and frames
N_frames = len(dataset.input)
N_cubes = np.size(np.unique(dataset.filenums))
nl = N_frames // N_cubes

In [ ]:
PSF_cube = dataset.psfs

In [ ]:
# parameters
pars = (56.790, 49.102)
planet_sep, planet_pa = pars
numbasis=[1,2,3,4,5,6,7,8,9,10,12,14,16,18,20]
num_k_klip = len(numbasis)
movement = 5
stamp_size = 10
plt.imshow(dataset.psfs[1])

In [ ]:
fm_class = es.ExtractSpec(dataset.input.shape,
                       numbasis,
                       planet_sep,
                       planet_pa,
                       PSF_cube,
                       np.unique(dataset.wvs),
                       stamp_size = stamp_size)

In [ ]:
fm.klip_dataset(dataset, fm_class,
                fileprefix="fmspect",
                annuli=[[planet_sep-stamp_size,planet_sep+stamp_size]],
                subsections=[[(planet_pa-stamp_size)/180.*np.pi,\
                              (planet_pa+stamp_size)/180.*np.pi]],
                movement=movement,
                numbasis = numbasis,
                spectrum=None,
                save_klipped=True, highpass=True,
                outputdir="output_path")

In [ ]:
klipped = dataset.fmout[:,:,-1,:]

In [ ]:
scale_factor = None
units = "natural"
exspect, fm_matrix = es.invert_spect_fmodel(dataset.fmout, dataset, units=units,
                                            scaling_factor=scale_factor,
                                            method="leastsq")
exspect

In [ ]:
#ERROR BAR CALCULATION
import pyklip.fakes as fakes

def fake_spect(pa, fake_flux, basis):
    psfs = np.tile(PSF_cube, (N_cubes, 1, 1))
    fakepsf = psfs * fake_flux[0,None,None]
    
    #tempdataset = charis.CHARISData(image_files,skipslices=[6,14,15,21])
    tempdataset = charis.CHARISData(image_files)
    fakes.inject_planet(tempdataset.input, tempdataset.centers, fakepsf, tempdataset.wcs, planet_sep, pa)
    
    fm_class = es.ExtractSpec(tempdataset.input.shape,
                       basis,
                       planet_sep,
                       pa,
                       PSF_cube,
                       np.unique(dataset.wvs),
                       stamp_size = stamp_size)
    
    fm.klip_dataset(tempdataset, fm_class,
                fileprefix="fmspect"+"pa_"+str(pa),
                annuli=[[planet_sep-stamp_size,planet_sep+stamp_size]],
                subsections=[[(pa-stamp_size)/180.*np.pi,\
                              (pa+stamp_size)/180.*np.pi]],
                movement=movement,
                numbasis = basis,
                spectrum=None,
                save_klipped=True, highpass=True,
                outputdir="output_path"+str(basis))
    
    exspect_fake, fm_matrix_fake = es.invert_spect_fmodel(tempdataset.fmout, tempdataset, units=units,
                                            scaling_factor=scale_factor,
                                            method="leastsq")
    
    del tempdataset
    return exspect_fake

npas = 11
pas = (np.linspace(planet_pa, planet_pa+360, num=npas+2)%360)[1:-1]
fake_spectra_all_bases = []
for i in range(len(numbasis)):
    input_spect = exspect[i,:]
    fake_spectra = np.zeros((npas, nl))
    for p, pa in enumerate(pas):
        fake_spectra[p,:] = fake_spect(pa, input_spect, numbasis[i])
    
    fake_spectra_all_bases.append(fake_spectra)

In [ ]:
from scipy import stats as spstat
error = []
for i in range(len(fake_spectra_all_bases)):
    for j in range(nl):
        x = fake_spectra_all_bases[i][:,j]
        err = spstat.iqr(x)
        error.append(err)
        print(err)

In [ ]:
e = np.array(error).reshape(int(len(error)/nl),nl)

In [ ]:
#FLUX CALIBRATION
import pysynphot as S
from uncertainties import ufloat
from uncertainties import unumpy
print(S)
current_dir = os.getcwd()
os.chdir("cdbs_path")

R_star = ufloat(2.29, 0.06) * 6.957 * (10**5)
D_star = ufloat(50.0, 0.1) * 3.086e13

#satellite spot to star ratio
spot_to_star_ratio = np.array([2.72*(10**-3)/(lamb/1.55)**2 for lamb in dataset.wvs[0:nl]])

#star spectrum
sp = S.Icat('ck04models', 11327, -0.36, 4.174)
sp.convert("micron")
sp.convert("mJy")
star_spectrum = np.array([sp.sample(dataset.wvs[i]) for i in range(nl)]) * (R_star / D_star)**2

#convert to ufloat
new_exspect = np.ndarray(shape=np.shape(exspect), dtype=object)
for i in range(len(exspect)):
    for j in range(nl):
        new_exspect[i][j] = ufloat(exspect[i,j],e[i,j])

#perform actual calibration
exspect_flux = new_exspect * star_spectrum * spot_to_star_ratio
#exspect_flux = exspect * star_spectrum * spot_to_star_ratio

#separate values
actual_values = np.zeros(np.shape(exspect))
error_bars = np.zeros(np.shape(exspect))
for i in range(len(exspect)):
    for j in range(nl):
        actual_values[i][j] = exspect_flux[i][j].nominal_value
        error_bars[i][j] = exspect_flux[i][j].std_dev


os.chdir("notebook_path")

In [ ]:
#create spectrum figure
import matplotlib.pyplot as plt
import math

    ref_wvs = [1.1596,1.1997,1.2412,1.2842,1.3286,1.3746,1.4222
           ,1.4714,1.5224,1.575,1.6296,1.686,1.7443,1.8047
           ,1.8672,1.9318,1.9987,2.0678,2.1394,2.2135,2.2901
           ,2.3693]

ref_valu = [0.591,0.6112,0.6942,0.8349,0.7658,0.2586,0.5371,0.583,
            0.8103,0.8248,1.1274,1.2744,1.019,0.8616,0.6344,0.975,
            0.8215,1.0233,1.2442,1.3643,1.2739,1.203]
ref_err = [0.0567,0.0529,0.0498,0.0487,0.0426,0.0403,0.0395,0.0393
           ,0.0397,0.0405,0.0375,0.0351,0.036,0.031,0.0508,0.0429
           ,0.036,0.0338,0.0477,0.0492,0.059,0.0832]


fig = plt.figure(figsize=(20,20), )
for i in range(len(numbasis)):
    ax = fig.add_subplot(math.ceil(len(numbasis)/4)+1,4,i+1)
    title = "Kappa And b Spectrum (KL={basis})"
    ax.set_title(title.format(basis=numbasis[i]))
    y_axis = "flux ({flux_units})"
    x_axis = "wavelength ({wv_units})"
    ax.set(xlabel=x_axis.format(wv_units=sp.waveunits), ylabel=y_axis.format(flux_units=sp.fluxunits))
    ax.errorbar(dataset.wvs[:nl], actual_values[i], yerr=error_bars[i], label='pyKLIP', capsize=3)
    ax.errorbar(ref_wvs, ref_valu, yerr=ref_err, label='Currie et al', capsize=3)
    ax.legend()
    
ax = fig.add_subplot(math.ceil(len(numbasis)/4) + 1,1,5)
ax.set_title("Kappa And b Spectra")
ax.set(xlabel=x_axis.format(wv_units=sp.waveunits), ylabel=y_axis.format(flux_units=sp.fluxunits))
for i in range(len(numbasis)):
    ax.plot(dataset.wvs[:nl], actual_values[i], label=str(numbasis[i]))
    ax.legend()

plt.tight_layout()

In [ ]:
#create magnitude comparison figure
ref_magnitudes = [11.58,10.70,10.05]
ref_error = [0.0985,0.086,0.086]
ref_wvs = np.array([1.250,1.635,2.150])
mag_val = new_exspect[:,[2,10,18]]
mag_wvs = np.array([dataset.wvs[2], dataset.wvs[10], dataset.wvs[18]])

mag_val = -2.5*unumpy.log10(mag_val*spot_to_star_ratio[[2,10,18]])

#separate values
actual_mag_val = np.zeros(np.shape(mag_val))
mag_error_bars = np.zeros(np.shape(mag_val))
for i in range(len(mag_val)):
    for j in range(3):
        actual_mag_val[i][j] = mag_val[i][j].nominal_value
        mag_error_bars[i][j] = mag_val[i][j].std_dev


fig, ax = plt.subplots()
ax.set_title(r'$\Delta$J, $\Delta$H, $\Delta$K Comparison')
ax.set(xlabel=x_axis.format(wv_units=sp.waveunits), ylabel="Difference in planet and star magnitude")
for i in range(len(numbasis)):
    ax.errorbar(mag_wvs, actual_mag_val[i], yerr=mag_error_bars[i], label='pyKLIP', capsize=3, color="orange")
ax.errorbar(ref_wvs, ref_magnitudes, yerr=ref_error, label='Currie et al', capsize=3)
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())
ax.set_ylim(9,13)

In [ ]:
#export spectra as csv
star_spect_val = []
star_spect_err = []
for i in range(len(star_spectrum)):
    star_spect_val.append(star_spectrum[i].nominal_value)
    star_spect_err.append(star_spectrum[i].std_dev)

data = {"wvs":dataset.wvs[:nl], "star_spect":star_spect_val, "star_err":star_spect_err}
for i in range(len(numbasis)):
    data[numbasis[i]] = exspect[i]

df = pd.DataFrame(data)
df.to_csv("output_path")

data = {"wvs":dataset.wvs[:nl]}
for i in range(len(numbasis)):
    data[numbasis[i]] = e[i]

df = pd.DataFrame(data)
df.to_csv("output_path")